# Model-Centric Federated Learning for Mobile - Spotify Recommendation

This notebook is adapted from the PySyft model-centric federated learning [MNIST example](https://github.com/OpenMined/PySyft/blob/syft_0.5.0/packages/syft/examples/federated-learning/model-centric/mcfl_create_plan_mobile.ipynb), modified for training a Spotify track recommendation model.

This is a simple mode. It will take in a user index and some features of the tracks such as tempo, and predict whether the user likes the track.

In [33]:
# stdlib
import base64
import json

# third party
import torch as th

# syft absolute
import syft as sy
from syft.core.plan.plan_builder import ROOT_CLIENT
from syft.core.plan.plan_builder import PLAN_BUILDER_VM
from syft.core.plan.plan_builder import make_plan
from syft.core.plan.translation.torchscript.plan_translate import (
    translate as translate_to_ts,
)
from syft.federated.model_centric_fl_client import ModelCentricFLClient
from syft.lib.python.int import Int
from syft.lib.python.list import List
from itertools import zip_longest

In [34]:
th.random.manual_seed(42)

## Model and Training Configurations

* n_users: number of users joining the training

In [35]:
# Parameters
n_users = 10
song_features = 10
bs = 64  # batch_size
lr = 5e-4
embedding_size = 50
# Number of layer is hard-coded, any changes on no. of layers would require change of the EmbeddingNet class too.
layer_sizes = [(embedding_size + song_features, 150), (150, 300), (300, 200), (200, 1)]


## Step 1: Define the model

Let's create a embedding net, which contains an embedding layer embedding user identify, and a fully connected net that has concat of embedding output and the song features as input.

In [36]:
class EmbeddingNet(
    sy.Module
):  # This version follows hinnes' implmentation (first assume only one user and using numeric attributes of the song)
    """
    Simple model with method for loss and hand-written backprop.
    """

    def __init__(self, torch_ref) -> None:
        super(EmbeddingNet, self).__init__(torch_ref=torch_ref)
        self.torch_ref = torch_ref
        self.embedlayer = torch_ref.nn.Linear(n_users, embedding_size)
        torch_ref.nn.init.constant_(self.embedlayer.bias.data, 0)
        self.embeddrop = torch_ref.nn.Dropout(0.02)
        self.fc1 = torch_ref.nn.Linear(layer_sizes[0][0], layer_sizes[0][1])
        self.a1 = torch_ref.nn.ReLU()
        self.d1 = torch_ref.nn.Dropout(0.3)
        self.fc2 = torch_ref.nn.Linear(layer_sizes[1][0], layer_sizes[1][1])
        self.a2 = torch_ref.nn.ReLU()
        self.d2 = torch_ref.nn.Dropout(0.3)
        self.fc3 = torch_ref.nn.Linear(layer_sizes[2][0], layer_sizes[2][1])
        self.a3 = torch_ref.nn.ReLU()
        self.fc4 = torch_ref.nn.Linear(layer_sizes[3][0], layer_sizes[3][1])

    def forward(self, users, features, x):
        """
        users: a one-hot tensor of size (n_users,) representing the user.
        features: 10d vector using spotify-provided feature values
        x: a 60d dummy vector required from user
        """
        self.embedout = self.embedlayer(users.float())
        self.embedout = self.embeddrop(self.embedout)
        x[:, :embedding_size] = self.embedout
        x[:, embedding_size : embedding_size + song_features] = features
        self.catout = x
        x = self.fc1(self.catout)
        x = self.a1(x)
        self.l1out = self.d1(x)
        y = self.fc2(self.l1out)
        y = self.a2(y)
        self.l2out = self.d2(y)
        z = self.fc3(self.l2out)
        self.l3out = self.a3(z)
        k = self.fc4(self.l3out)
        self.l4out = self.torch_ref.sigmoid(k)
        return self.l4out

    def backward(self, user, error):  # BATCH_SIZE=1 is assumed.
        pgs, flayers, aouts = [], [], []
        latest_grad = error
        aouts.extend([self.catout, self.l1out, self.l2out, self.l3out])
        flayers.extend([self.fc2, self.fc3, self.fc4])
        for i in range(len(aouts)):
            j = len(aouts) - i - 1
            pgs.append(latest_grad.sum(0))  # bias grad
            pgs.append(latest_grad.t() @ aouts[j])  # weight grad
            if j - 1 >= 0:
                latest_grad = (latest_grad @ flayers[j - 1].state_dict()["weight"]) * (
                    aouts[j] > 0
                ).float()
            else:  # Embedding layer
                latest_grad = (
                    latest_grad @ self.fc1.state_dict()["weight"]
                )  # no ReLU in embedding
                latest_grad = latest_grad[:, :embedding_size]
        embedgrad = latest_grad.t() @ user.float()
        # For embedding layer, we mimic using a linear layer.
        # Therefore, bias exists but we do not need it.
        # Hence providing a zero tensor so that the bias weight remains 0.
        pgs.append(th.zeros((embedding_size,)))
        pgs.append(embedgrad)
        pgs.reverse()
        return tuple(pgs)

    def mse_loss(self, fpass, target):  # MSE first
        squared_error = self.torch_ref.pow(fpass - target, 2)
        loss = squared_error.mean()
        loss_grad = (target - fpass) * fpass * (1 - fpass) / bs
        return loss, loss_grad


In [37]:
model = EmbeddingNet(th)

features = th.randn(bs, song_features)
users = th.randint(1, n_users, (bs,))
users = th.nn.functional.one_hot(users)
x = th.randn(bs, song_features + embedding_size)
fpass = model(users, features, x)
batch_size = th.tensor([bs])
ys = th.rand(bs, 1)
loss, loss_grad = model.mse_loss(fpass, ys)
print("Results here:")
grads = model.backward(users, loss_grad)
# print(grads.shape)
for x in grads:
    print(x.shape)
print("Results sdf here:")
for y in model.parameters():
    print(y.shape)
# updated_params = tuple(
#         param - lr * grad for param, grad in zip(model.get_model_parameters(), grads)
#     )
# print(model.fc2.state_dict()["weight"].shape)


Results here:
torch.Size([50, 10])
torch.Size([50])
torch.Size([150, 60])
torch.Size([150])
torch.Size([300, 150])
torch.Size([300])
torch.Size([200, 300])
torch.Size([200])
torch.Size([1, 200])
torch.Size([1])
Results sdf here:
torch.Size([50, 10])
torch.Size([50])
torch.Size([150, 60])
torch.Size([150])
torch.Size([300, 150])
torch.Size([300])
torch.Size([200, 300])
torch.Size([200])
torch.Size([1, 200])
torch.Size([1])


## Step 2: Define Training Plan

In [38]:
def set_remote_model_params(module_ptrs, params_list_ptr):
    """Sets the model parameters into traced model"""
    param_idx = 0
    for module_name, module_ptr in module_ptrs.items():
        for param_name, _ in PLAN_BUILDER_VM.store[
            module_ptr.id_at_location
        ].data.named_parameters():
            module_ptr.register_parameter(param_name, params_list_ptr[param_idx])
            param_idx += 1

# Create the model
local_model = EmbeddingNet(th)


model_params_zeros = sy.lib.python.List(
    [th.nn.Parameter(th.zeros_like(param)) for param in local_model.parameters()]
)

@make_plan
def training_plan(
    features = th.randn(bs,10),
    users = th.nn.functional.one_hot(th.randint(1,n_users,(bs,))),
    xs = th.randn(bs,embedding_size+song_features),
    ys=th.rand(bs,1),
    lr=th.tensor([lr]),
    params=model_params_zeros,
):
    # send the model to plan builder (but not its default params)
    # this is required to build the model inside the Plan
    model = local_model.send(ROOT_CLIENT, send_parameters=False)

    # set model params from input
    set_remote_model_params(model.modules, params)
    

    # forward
    fpass = model(users,features,xs)

    # loss
    loss, loss_grad = model.mse_loss(fpass,ys)

    # backward
    grads = model.backward(users, loss_grad)

    # SGD step
    updated_params = tuple(
        param - lr * grad for param, grad in zip(model.parameters(), grads)
    )

    # return things
    return (loss, *updated_params)

Translate the training plan to torchscript so it can be used with mobile workers.

In [39]:
# Translate to torchscript
ts_plan = translate_to_ts(training_plan)

# Let's examine its contents
print(ts_plan.torchscript.code)

/home/hinnes/projects/pysyft-spotify-recommendation/env/lib/python3.8/site-packages/torch/tensor.py:587: RuntimeWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  warnings.warn('Iterating over a tensor might cause the trace to be incorrect. '
/home/hinnes/projects/pysyft-spotify-recommendation/env/lib/python3.8/site-packages/syft/lib/torch/tensor_util.py:53: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  numpy_tensor = tensor.detach().numpy()
/home/hinnes/projects/pysyft-spotify-recommendation/env/lib/python3.8/site-packages/syft/lib/torch/tensor_util.py:105: TracerWarning: torch.from_numpy results are regi

def forward(self,
    tensor: Tensor,
    tensor0: Tensor,
    input: Tensor,
    tensor1: Tensor,
    tensor2: Tensor,
    argument_6: List[Tensor]) -> List[Tensor]:
  tensor3, tensor4, weight, tensor5, weight0, tensor6, weight1, tensor7, tensor8, tensor9, = argument_6
  input0 = torch.to(tensor0, 6, False, False, None)
  input1 = torch.addmm(tensor4, input0, torch.t(tensor3), beta=1, alpha=1)
  _0 = torch.dropout(input1, 0.019999999552965164, True)
  _1 = torch.slice(input, 0, 0, 9223372036854775807, 1)
  _2 = torch.copy_(torch.slice(_1, 1, 0, 50, 1), _0, False)
  _3 = torch.slice(input, 0, 0, 9223372036854775807, 1)
  _4 = torch.copy_(torch.slice(_3, 1, 50, 60, 1), tensor, False)
  input2 = torch.addmm(tensor5, input, torch.t(weight), beta=1, alpha=1)
  input3 = torch.relu(input2)
  input4 = torch.dropout(input3, 0.30000001192092896, True)
  input5 = torch.addmm(tensor6, input4, torch.t(weight0), beta=1, alpha=1)
  input6 = torch.relu(input5)
  input7 = torch.dropout(input6, 0.30000

/home/hinnes/projects/pysyft-spotify-recommendation/env/lib/python3.8/site-packages/torch/jit/_trace.py:934: TracerWarning: Encountering a list at the output of the tracer might cause the trace to be incorrect, this is only valid if the container structure does not change based on the module's inputs. Consider using a constant container instead (e.g. for `list`, use a `tuple` instead. for `dict`, use a `NamedTuple` instead). If you absolutely need this and know the side effects, pass strict=False to trace() to allow this behavior.
  module._c._create_method_from_trace(


## Step 3: Define Averaging Plan

Averaging Plan is executed by PyGrid at the end of the cycle,
to average _diffs_ submitted by workers and update the model
and create new checkpoint for the next cycle.

_Diff_ is the difference between client-trained
model params and original model params,
so it has same number of tensors and tensor's shapes
as the model parameters.

We define Plan that processes one diff at a time.
Such Plans require `iterative_plan` flag set to `True`
in `server_config` when hosting FL model to PyGrid.

Plan below will calculate simple mean of each parameter.

In [40]:
@make_plan
def avg_plan(
    avg=List(local_model.parameters()), item=List(local_model.parameters()), num=Int(0)
):
    new_avg = []
    for i, param in enumerate(avg):
        new_avg.append((avg[i] * num + item[i]) / (num + 1))
    return new_avg

## Step 4: Define Federated Learning Process Configuration

Before hosting the model and training plan to PyGrid,
we need to define some configuration parameters, such as
FL process name, version, workers configuration,
authentication method, etc.

In [41]:
name = "spotify_recommendation"
version = "1.0"

client_config = {
    "name": name,
    "version": version,
    "batch_size": bs,
    "lr": lr,
    "max_updates": 100,  # custom syft.js option that limits number of training loops per worker
}

server_config = {
    # "min_workers": 2,
    # "max_workers": 2,
    # "pool_selection": "random",
    # "do_not_reuse_workers_until_cycle": 6,
    "cycle_length": 28800,  # max cycle length in seconds
    "num_cycles": 30,  # max number of cycles
    "max_diffs": 1,  # number of diffs to collect before avg
    "minimum_upload_speed": 0,
    "minimum_download_speed": 0,
    "iterative_plan": True,  # tells PyGrid that avg plan is executed per diff
}

This FL process will require workers to authenticate with signed JWT token.
Providing the `pub_key` in FL configuration allows PyGrid to verify JWT tokens.

In [42]:
def read_file(fname):
    with open(fname, "r") as f:
        return f.read()

In [43]:
public_key = read_file("example_rsa.pub").strip()

server_config["authentication"] = {
    "type": "jwt",
    "pub_key": public_key,
}

## Step 5: Host in PyGrid

Let's now host everything in PyGrid so that it can be accessed by worker libraries.

Note: assuming the PyGrid Domain is running locally on port 7001.

In [44]:
grid_address = "localhost:7001"

### Setup the domain

Run this once and only once after the PyGrid is cleared.

In [45]:
from syft.grid.client.client import connect
from syft.grid.client.grid_connection import GridHTTPConnection

domain = connect(
    url=f"http://{grid_address}", 
    conn_type=GridHTTPConnection,
)
domain.setup(
    email="owner@openmined.org",
    password="owerpwd",
    domain_name="OpenMined Node",
    token="9G9MJ06OQH",
)

In [46]:
grid = ModelCentricFLClient(address=grid_address, secure=False)
domain = grid.connect()

Following code sends FL model, training plans, and configuration to the PyGrid:

In [47]:
response = grid.host_federated_training(
    model=local_model,
    client_plans={
        # Grid can store both types of plans (regular for python worker, torchscript for mobile):
        "training_plan": training_plan,
        "training_plan:ts": ts_plan,
    },
    client_protocols={},
    server_averaging_plan=avg_plan,
    client_config=client_config,
    server_config=server_config,
)

In [48]:
response

{'type': 'model-centric/host-training', 'data': {'status': 'success'}}

If you see successful response, you've just hosted your first FL process into PyGrid!

If you see error that FL process already exists,
this means FL process with such name and version is already hosted.
You might want to update name/version in configuration above, or cleanup PyGrid database.

To cleanup database, set path below correctly and run:

In [49]:
# !rm PyGrid/apps/domain/src/nodedatabase.db


To train hosted model, use one of the existing mobile FL workers:
 * [SwiftSyft](https://github.com/OpenMined/SwiftSyft) (see included worker example)
 * [KotlinSyft](https://github.com/OpenMined/KotlinSyft) (see included worker example)

Support for javascript worker is coming soon:
 * [syft.js](https://github.com/OpenMined/syft.js)
